### Test of CV functionality. 

Most storage related tests are nosetests now. So look in `testcollectivevariable.py` please.

In [1]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

Load first frame from test pdb.

In [2]:
template = peng.snapshot_from_pdb('../resources/AD_initial_frame.pdb')

Create a simple CV without an underlying function.

In [3]:
cv0 = paths.CollectiveVariable('func0')

Create a complicated function.

In [4]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [5]:
cv1 = paths.FunctionCV('func1', dist, center=center, np=np, cv_time_reversible=False).with_diskcache()
cv2 = paths.FunctionCV('func2', dist, center=center, np=np, cv_wrap_numpy_array=True).with_diskcache()
cv3 = paths.FunctionCV('func3', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True).with_diskcache()

Create storage to test save and load.

In [6]:
#! lazy
storage = paths.Storage('can_be_deleted.nc', mode='w')
print storage.snapshots.save(template)

124570917265566791587849024337315102732


Save CV

In [7]:
#! lazy
print storage.save([cv0, cv1, cv2, cv3])

[(store.cvs[Attribute] : 4 object(s), 4, 124570917265566791587849024337315102734L), (store.cvs[Attribute] : 4 object(s), 4, 124570917265566791587849024337315102736L), (store.cvs[Attribute] : 4 object(s), 4, 124570917265566791587849024337315102738L), (store.cvs[Attribute] : 4 object(s), 4, 124570917265566791587849024337315102740L)]


In [8]:
storage.cvs.index

{124570917265566791587849024337315102734L: 0,
 124570917265566791587849024337315102736L: 1,
 124570917265566791587849024337315102738L: 2,
 124570917265566791587849024337315102740L: 3}

Set the CV value for the storage.template

In [9]:
cv0[template] = 10.0

and create a storage for it

In [10]:
storage.cvs.add_diskcache(cv0, allow_incomplete=True)

And we should have a float store

In [11]:
storage.cvs.sync(cv0)

In [12]:
assert(cv0._store_dict.value_store.vars['value'][:] == [10.0])

Test function for reversed template.

In [13]:
dd = dist(template.reversed, center, np)
print dd

-1.08320000023


Create another CV. This time using the `from_template` function.

In [14]:
print cv0([template, template])
print cv1([template, template])

[10.0, 10.0]
[-1.0832000002264977, -1.0832000002264977]


In [15]:
print type(cv0([template, template]))
print type(cv0([template, template])[0])
print type(cv1([template, template]))
print type(cv1([template, template])[0])

<type 'list'>
<type 'float'>
<type 'list'>
<type 'numpy.float64'>


In [16]:
#! skip
print storage.cvs.variables['json'][:]

[ u'{"_cls":"CollectiveVariable","_dict":{"cv_time_reversible":false,"name":"func0"}}'
 u'{"_cls":"FunctionCV","_dict":{"name":"func1","f":{"_marshal":"YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTQtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB","_module_vars":[],"_global_vars":[]},"cv_time_reversible":false,"cv_wrap_numpy_array":false,"cv_requires_lists":false,"cv_scalarize_numpy_singletons":false,"kwargs":{"np":{"_import":"numpy"},"center":1}}}'
 u'{"_cls":"FunctionCV","_dict":{"name":"func2","f":{"_marshal":"YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTQtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB","_module_vars":[],"_global_vars":[]},

In [17]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]
cv3j = storage.cvs.vars['json'][3]

In [18]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv3j([template, template, template])
assert(res[0] == res[1] == res[2])

In [19]:
t = paths.Trajectory([template] * 4)

In [20]:
storage.save(t)

(store.trajectories[Trajectory] : 1 object(s),
 1,
 124570917265566791587849024337315102810L)

In [21]:
def ff(t, cv3):
    return max(cv3(t))

In [22]:
a = paths.netcdfplus.FunctionAttribute('max_cv3', paths.Trajectory, ff, cv3=cv3)

In [23]:
storage.attributes.save(a)

124570917265566791587849024337315102812L

In [24]:
a(t)

-1.0832000002264977

In [25]:
p = paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__)

In [26]:
a(paths.netcdfplus.LoaderProxy(storage.trajectories, t.__uuid__))

-1.0832000002264977

In [27]:
storage.trajectories.add_attribute(paths.netcdfplus.ValueStore, a, t, allow_incomplete=False)

store.trajectories_max_cv3[None/ANY] : (not created)

In [28]:
ats = storage.trajectories.attribute_list[a]

In [29]:
print ats.vars['value'][:]
if ats.allow_incomplete:
    print ats.vars['index'][:]

[-1.0832]


In [30]:
a(t)

-1.0832000002264977

In [31]:
print a._store_dict.value_store.vars['value'][:]

[-1.0832]


In [32]:
a._store_dict.value_store.variables['value']

<type 'netCDF4._netCDF4.Variable'>
float64 trajectories_max_cv3_value(trajectories)
    var_type: numpy.float64
unlimited dimensions: trajectories
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

In [33]:
cv3._store_dict.value_store.variables['value']

<type 'netCDF4._netCDF4.Variable'>
float64 cv3_value(snapshots)
    var_type: numpy.float64
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

In [42]:
storage.trajectories.sync_attribute(a)

In [38]:
storage.attributes.set_cache_store(a)

In [35]:
a._store_dict

In [36]:
storage.attributes.has_cache(a)

True

In [34]:
storage.attributes.sync(a)

In [1]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

In [2]:
st = paths.Storage('../alanine_dipeptide_mstis/ala_mstis_production.nc', mode='r')

In [3]:
st.trajectories

store.trajectories[Trajectory] : 34 object(s)

In [4]:
st.vars['cvcache'][:]

[store.cv1[None/ANY] : 2135 object(s), store.cv2[None/ANY] : 2135 object(s)]

In [5]:
st.trajectories[0]

Trajectory[22]

In [13]:
cv = st.cvs[2]

In [22]:
cv(st.trajectories.last)

array([-1.2482394 , -1.4427911 , -1.61954296, -1.589432  , -1.64347446,
       -1.50749338, -1.6607753 , -1.62424457, -1.70517468, -1.89886057,
       -2.03728652, -1.92859757, -1.82200861, -1.69064188, -1.76828671,
       -1.66275394, -1.82730448, -1.84174562, -1.71697426, -1.67712629,
       -1.70510352, -1.53854132, -1.47162092, -1.58586621, -1.71889877,
       -1.64669454, -1.8108778 , -1.74472785, -1.59468865, -1.7160939 ,
       -1.96709645, -1.86877465, -1.66174531, -2.02366853, -2.04881477,
       -2.06649089, -1.6752913 , -1.54426301, -1.72473848, -1.63674879,
       -1.60655177, -1.70646822, -1.72454131, -1.87965143, -1.78662598,
       -1.86745083, -1.86155272, -2.36759877, -2.22496247, -1.83800161,
       -2.07135463, -1.99307215, -2.21489692, -2.31013966, -1.91558814,
       -1.72727311, -1.7184273 , -1.90977263, -1.66819334, -1.72114098,
       -1.64033759, -1.58776546, -1.62051415, -1.5337224 , -1.67006946,
       -1.67202127, -1.57419062, -1.55832827, -1.53306866, -1.08

In [24]:
max_vals = [paths.netcdfplus.FunctionAttribute(cv.name + '_max', paths.Trajectory, lambda t, cv: max(cv(t)), cv=cv) for cv in st.cvs]

In [31]:
max_vals[0](st.trajectories)

[46.4351040506785,
 191.24625976357635,
 182.10571754327745,
 180.32425213350996,
 175.04995035215285,
 175.04995035215285,
 186.9394812672714,
 178.56667188056713,
 175.04995035215285,
 181.9019221320897,
 26.28924700416413,
 179.50846120778468,
 155.47120330946626,
 175.04995035215285,
 186.9394812672714,
 177.65417940829713,
 179.81444691049407,
 186.9394812672714,
 180.04547406963445,
 186.9394812672714,
 181.6274130613759,
 29.07434357276598,
 27.15438809392325,
 186.9394812672714,
 177.6380723503342,
 181.35276408951677,
 179.50846120778468,
 183.90180065452628,
 175.04995035215285,
 38.48918639297213,
 177.76069778069564,
 180.86776583227055,
 106.04335302715421,
 155.47120330946626]

In [28]:
st.cvs[0](st.trajectories[0])

[6.478404851516733,
 16.037696953677592,
 19.088752635815656,
 23.283941813216337,
 21.560290689422153,
 18.3380813143434,
 20.762718778209166,
 21.266532768965885,
 16.293838594523898,
 19.622974445792167,
 11.148704708012408,
 33.38987639705878,
 28.48975196775723,
 30.120026014131092,
 30.20375435461709,
 38.48918639297213,
 34.792877580196695,
 12.951992387070552,
 46.4351040506785,
 15.178670104631703,
 13.874227347138572,
 8.44582675258491]

In [32]:
a = max_vals[0]

In [33]:
a.with_diskcache()

In [34]:
a.diskcache_allow_incomplete

False